<a href="https://colab.research.google.com/github/gitconnoisseur/toxicity-model/blob/master/ecfp_xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# For installation on Colab:
!pip install PyTDC
!pip install deepchem
!pip install autokeras
!pip install --upgrade numpy

In [2]:
#import autogluon as ag
from tdc.single_pred import Tox
import deepchem
import xgboost

import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak

from tdc.benchmark_group import admet_group
group = admet_group(path = 'data/')

Found local copy...


Run benchmark using a GCN with deepchem's `ConvMolFeaturizer`:

In [6]:
predictions_list = []
metric = deepchem.deepchem.metrics.Metric(deepchem.deepchem.metrics.mean_absolute_error)
featurizer = deepchem.feat.ConvMolFeaturizer()

for seed in [1, 2, 3, 4, 5]:
    benchmark = group.get('LD50_Zhu') 
    
    predictions = {}
    name = benchmark['name']
    train_val, test = benchmark['train_val'], benchmark['test']
    train, valid = group.get_train_valid_split(benchmark = name, split_type = 'default', seed = seed)  

    #featurize training, valid, and test data
    f_train = featurizer.featurize(train.iloc[:,1].to_list())
    f_valid = featurizer.featurize(valid.iloc[:,1].to_list())
    f_test = featurizer.featurize(test.iloc[:,1].to_list())

    #convert training and validation data into a deepchem dataset
    train_data = deepchem.deepchem.data.NumpyDataset(X=f_train, y=np.array(train.iloc[:,2]), ids=np.array(train.iloc[:,1].to_list()))
    valid_data = deepchem.deepchem.data.NumpyDataset(X=f_valid, y=np.array(valid.iloc[:,2]), ids=np.array(valid.iloc[:,1].to_list()))

    #fit data
    reg = deepchem.deepchem.models.GraphConvModel(
        n_tasks=1, 
        dropout=.0005,
        dense_layer_size=1063,
        graph_conv_layers=[128, 128, 128],
        mode="regression",)
    callback = deepchem.deepchem.models.ValidationCallback(valid_data, 1000, metric)
    reg.fit(train_data, nb_epoch=100, callbacks=callback)

    # store test predictions in y_pred_test
    y_pred_test = reg.predict_on_batch(X=np.array(f_test))

    print("predictions for run #")
    print(seed)
    print(y_pred_test[0:5])
        
    predictions[name] = y_pred_test
    predictions_list.append(predictions)

print("Prediction List:")
print(predictions_list)

generating training, validation splits...
100%|██████████| 5907/5907 [00:01<00:00, 4475.87it/s]


Step 1000 validation: mean_absolute_error=0.60425
Step 2000 validation: mean_absolute_error=0.550885
Step 3000 validation: mean_absolute_error=0.564119
Step 4000 validation: mean_absolute_error=0.563506
Step 5000 validation: mean_absolute_error=0.568418


generating training, validation splits...


predictions for run #
1
[[1.934698 ]
 [3.2687738]
 [3.2575288]
 [4.4828014]
 [4.684982 ]]


100%|██████████| 5907/5907 [00:01<00:00, 4545.30it/s]


...

Step 1000 validation: mean_absolute_error=0.592864
Step 2000 validation: mean_absolute_error=0.583056
Step 3000 validation: mean_absolute_error=0.570302
Step 4000 validation: mean_absolute_error=0.559017
Step 5000 validation: mean_absolute_error=0.568747


generating training, validation splits...


predictions for run #
4
[[1.8217533]
 [2.7123652]
 [2.8046231]
 [3.7566226]
 [4.2453084]]


100%|██████████| 5907/5907 [00:01<00:00, 5026.17it/s]


Step 1000 validation: mean_absolute_error=0.53904
Step 2000 validation: mean_absolute_error=0.530885
Step 3000 validation: mean_absolute_error=0.527638
Step 4000 validation: mean_absolute_error=0.526491
predictions for run #
5
[[2.3002398]
 [2.9521763]
 [3.4880524]
 [4.057559 ]
 [4.450518 ]]
Prediction List:
[{'ld50_zhu': array([[1.934698 ],
       [3.2687738],
       [3.2575288],
       ...,
       [2.7233386],
       [1.809543 ],
       [2.6535265]], dtype=float32)}, {'ld50_zhu': array([[2.2235868],
       [2.837673 ],
       [3.4405148],
       ...,
       [1.9210163],
       [1.8645542],
       [2.745737 ]], dtype=float32)}, {'ld50_zhu': array([[1.897886 ],
       [3.0458996],
       [2.994091 ],
       ...,
       [2.105375 ],
       [2.0407841],
       [2.5119348]], dtype=float32)}, {'ld50_zhu': array([[1.8217533],
       [2.7123652],
       [2.8046231],
       ...,
       [2.8501215],
       [2.1627717],
       [2.596031 ]], dtype=float32)}, {'ld50_zhu': array([[2.3002398],
    

In [7]:
for i in predictions_list:
    print(group.evaluate(i))

group.evaluate_many(predictions_list)

{'ld50_zhu': {'mae': 0.623}}
{'ld50_zhu': {'mae': 0.652}}
{'ld50_zhu': {'mae': 0.639}}
{'ld50_zhu': {'mae': 0.631}}
{'ld50_zhu': {'mae': 0.595}}


{'ld50_zhu': [0.628, 0.019]}

Evaluate optimal hyperparameters using R^2 score because the module tries to maximize the metric:

In [14]:
hyperOptMetric = deepchem.deepchem.metrics.Metric(deepchem.deepchem.metrics.r2_score)

def model_builder(**model_params):
  return deepchem.deepchem.models.GraphConvModel(n_tasks=1, mode="regression", graph_conv_layers=[128, 128, 128])

#optimizer = deepchem.deepchem.hyper.GaussianProcessHyperparamOpt(model_builder)

optimizer = deepchem.deepchem.hyper.GaussianProcessHyperparamOpt(model_builder)

params_dict = {"dropout": .01, "dense_layer_size": 200}

best_model, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict, train_data, valid_data, hyperOptMetric, search_range=10)

Evaluation 	 Proposed point 	  Current eval. 	 Best eval.
init   	 [9.54472202e-02 1.66300000e+03]. 	  0.05614210575885126 	 0.13022634235269537
init   	 [7.81778992e-02 1.95200000e+03]. 	  0.11803599063732828 	 0.13022634235269537
init   	 [6.78009843e-02 3.88000000e+02]. 	  0.13022634235269537 	 0.13022634235269537
1      	 [1.21594231e-02 4.02000000e+02]. 	  0.10297839213545745 	 0.13022634235269537
2      	 [5.63577392e-02 1.94996902e+03]. 	  0.07543380244710662 	 0.13022634235269537
3      	 [8.73771372e-02 3.99869355e+02]. 	  0.10647494762543674 	 0.13022634235269537
4      	 [5.73742248e-02 3.86042513e+02]. 	  -0.015072240034492701 	 0.13022634235269537
5      	 [1.44460922e-02 1.93100000e+03]. 	  0.04720652864226782 	 0.13022634235269537
6      	 [6.92807233e-02 1.93369230e+03]. 	  0.21513004052653117 	 0.21513004052653117
7      	 [1.00000000e-01 3.89993332e+02]. 	  0.08601514697821022 	 0.21513004052653117
8      	 [5.25671057e-03 1.94747276e+03]. 	  0.09740603500068701 	 0.2

In [15]:
print(best_hyperparams)

{'dropout': 0.0661305398871231, 'dense_layer_size': 397}
